In [0]:
/* 03_gold_claims_by_hour.ipynb
GOAL 1: Hourly claim volume and spend (count, sum, avg)
GOAL 2: Daily provider spend with 7-day rolling totals (window function demo)
GOAL 3: QA view — hourly provider match rate

SOURCE: kardia_silver.silver_claims_enriched
OUTPUT: TABLE: kardia_gold.gold_hourly_claim_metrics
        TABLE: kardia_gold.gold_provider_7d_spend
        VIEW : kardia_gold.gold_hourly_match_qc_vw

TRIGGER: Full snapshot overwrite each run — fast and simple for small datasets.
         Use foreachBatch and MERGE in production to avoid rewriting unchanged data.

NOTE: The Gold Claims/Providers tables use Delta Liquid Clustering and Predictive Optimization
      to reduce small files and maintain layout efficiency without manual OPTIMIZE or ZORDER. */

In [0]:
%sql
-- 1. Ensure database exists
CREATE DATABASE IF NOT EXISTS kardia_gold;

-- 2. Create hourly claim metrics table (CTAS)
    CREATE OR REPLACE TABLE kardia_gold.gold_hourly_claim_metrics AS
    SELECT DATE_TRUNC('hour', _ingest_ts) AS hour_ts,
           COUNT(*) AS claim_cnt,
           SUM(ClaimAmount) AS total_amount,
           AVG(ClaimAmount) AS avg_amount,
           SUM(IF(ProviderSpecialty IS NULL, 1, 0)) AS unmatched_cnt
    FROM kardia_silver.silver_claims_enriched
    GROUP BY DATE_TRUNC('hour', _ingest_ts);

-- Enable Predictive Optimization
    ALTER TABLE kardia_gold.gold_hourly_claim_metrics
    SET TBLPROPERTIES (
      'delta.autoOptimize.optimizeWrite' = 'true',
      'delta.autoOptimize.autoCompact' = 'true'
    );

In [0]:
%sql
-- Aggregate claim amounts by provider and date,
--  then calculate 7-day rolling totals and averages using window functions.
CREATE OR REPLACE TABLE kardia_gold.gold_provider_7d_spend AS
    WITH daily_spend AS (
        SELECT
            TO_DATE(ClaimDate) AS claim_date,
            ProviderID,
            SUM(ClaimAmount)   AS daily_amount
        FROM kardia_silver.silver_claims_enriched
        GROUP BY TO_DATE(ClaimDate), ProviderID
    )
    SELECT
        claim_date,
        ProviderID,
        daily_amount,
        SUM(daily_amount) OVER (
            PARTITION BY ProviderID
            ORDER BY claim_date
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ) AS rolling_7d_total,
        AVG(daily_amount) OVER (
            PARTITION BY ProviderID
            ORDER BY claim_date
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ) AS rolling_7d_avg
    FROM daily_spend;

-- Enable Predictive Optimization
    ALTER TABLE kardia_gold.gold_provider_7d_spend
    SET TBLPROPERTIES (
      'delta.autoOptimize.optimizeWrite' = 'true',
      'delta.autoOptimize.autoCompact' = 'true'
    );

In [0]:
%sql
/* Define a QA view that calculates the hourly provider match rate
   by dividing matched claims by total claims from the hourly metrics. */
   
    CREATE OR REPLACE VIEW kardia_gold.gold_hourly_match_qc_vw AS
    SELECT hour_ts,
           claim_cnt,
           unmatched_cnt,
           ROUND(1 - unmatched_cnt / claim_cnt, 4) AS match_rate
    FROM kardia_gold.gold_hourly_claim_metrics;


In [0]:
%sql
-- Preview: Top 10 rows from hourly claim metrics
SELECT * FROM kardia_gold.gold_hourly_claim_metrics
ORDER BY hour_ts DESC
LIMIT 10;

-- Preview: Top 10 rows from provider 7-day spend
SELECT * FROM kardia_gold.gold_provider_7d_spend
ORDER BY claim_date DESC, ProviderID
LIMIT 10;

-- Preview: Top 10 rows from hourly provider match QA view
SELECT * FROM kardia_gold.gold_hourly_match_qc_vw
ORDER BY hour_ts DESC
LIMIT 10;